# PC2Beam Slim Demonstrator - Google Colab

This notebook demonstrates the PC2Beam point cloud to beam processing pipeline.

**To use this demonstrator:**
1. Click "Runtime" → "Run all" to execute all cells
2. Use the interactive widgets to select datasets and process them
3. View results and visualizations

**Note:** This runs entirely in your browser - no installation required!

In [ ]:
# Install required packages
!pip install plotly ipywidgets numpy pandas scikit-learn

In [ ]:
# Import everything from the demo module
from pc2beam.demo import (
    generate_dummy_data,
    process_pipeline,
    create_3d_beam_projection,
    create_xy_scatter_plot
)
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

print('PC2Beam demonstrator loaded successfully!')

In [ ]:
# Generate datasets and create widgets
datasets = generate_dummy_data()
print(f'Generated {len(datasets)} datasets')

# Create interactive widgets
dataset_options = [(f'{name} ({len(data["points"])} points)', name) for name, data in datasets.items()]
dataset_dropdown = widgets.Dropdown(options=dataset_options, value=dataset_options[0][1], description='Dataset:')
voxel_size = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='Voxel Size:')
min_points = widgets.IntSlider(value=10, min=1, max=100, description='Min Points:')
process_button = widgets.Button(description='🚀 Process Point Cloud', button_style='primary')
output_area = widgets.Output()
progress_bar = widgets.FloatProgress(value=0, min=0, max=1, description='Progress:')
progress_bar.layout.display = 'none'
status_text = widgets.HTML(value='<p style="color: blue;">Ready to process</p>')

In [ ]:
# Processing function
def process_pointcloud(b):
    with output_area:
        clear_output(wait=True)
        dataset_name = dataset_dropdown.value
        if not dataset_name:
            status_text.value = '<p style="color: red;">Please select a dataset</p>'
            return
        
        points = datasets[dataset_name]['points']
        status_text.value = '<p style="color: orange;">Processing...</p>'
        progress_bar.layout.display = 'block'
        
        try:
            # Run complete pipeline
            results = process_pipeline(points, voxel_size.value, min_points.value)
            progress_bar.value = 1.0
            
            # Display results
            print(f'📁 Dataset: {dataset_name}')
            print(f'📊 Original points: {len(points):,}')
            print(f'🔍 S1 features: {len(results["s1_features"]):,}')
            print(f'📈 S2 features: {len(results["s2_features"]):,}')
            
            # Create visualizations
            fig_3d = create_3d_beam_projection(results['beam_projection'])
            fig_3d.update_layout(title=f'3D Beam Projection - {dataset_name}')
            fig_3d.show()
            
            fig_xy = create_xy_scatter_plot(results['beam_projection'])
            fig_xy.update_layout(title=f'XY Projection - {dataset_name}')
            fig_xy.show()
            
            # Feature statistics
            if results['s1_features']:
                s1_df = pd.DataFrame(results['s1_features'])
                print('\n📊 S1 Feature Statistics:')
                display(s1_df.describe())
            
            if results['s2_features']:
                s2_df = pd.DataFrame(results['s2_features'])
                print('\n📊 S2 Feature Statistics:')
                display(s2_df.describe())
            
            status_text.value = '<p style="color: green;">🎉 Processing completed!</p>'
            
        except Exception as e:
            print(f'❌ Error: {str(e)}')
            status_text.value = f'<p style="color: red;">❌ Error: {str(e)}</p>'
        finally:
            progress_bar.layout.display = 'none'

process_button.on_click(process_pointcloud)

In [ ]:
# Display interface
controls = widgets.VBox([
    widgets.HTML(value='<h3>📁 Dataset Selection</h3>'),
    dataset_dropdown,
    widgets.HTML(value='<h3>⚙️ Processing Parameters</h3>'),
    voxel_size,
    min_points,
    widgets.HTML(value='<h3>🚀 Actions</h3>'),
    process_button,
    status_text,
    progress_bar
])
display(controls)
display(output_area)

## 📖 Instructions

1. **Select a dataset** from the dropdown menu
2. **Adjust parameters** if needed:
   - **Voxel Size**: Controls feature extraction resolution
   - **Min Points**: Minimum points per voxel
3. **Click 'Process Point Cloud'** to run the pipeline
4. **View results** including 3D visualizations and statistics

## 🔬 Pipeline Stages

1. **S1 Features**: Geometric features from voxelized point clouds
2. **S2 Features**: Statistical features describing point distributions
3. **Beam Projection**: Mapping 3D points to 2D beam coordinates

## 🌐 Sharing

Save to Google Drive and share the notebook link - runs in any browser!